## Decision Model Testing 
Preliminaries for .toml files 

In [ ]:
cd("../TestDrive")

In [ ]:
] activate .

In [ ]:
using JuMP, MathOptInterface, Clp, Ipopt, GLPK

In [ ]:
using PyPlot
#using Plots
#gr()
; 

In [ ]:
] up 

In [ ]:
include("src/TestDrive.jl")

### Create scenario
To invoke the structs and data used in the popgen model for a single node

In [ ]:
# REMOVE MIGRATION FROM NODES FOR FUTURE USE - 
# SHOULD ONLY OCCUR IN NETWORK STRUCT: 
migrate1 = rand((6*(4+8+6+1+6)), 2, 2) 

# User-named node  
firstnode = :FirstNode

# User-constructed node: (name, gene_data, stages, location, migration_matrix)
node = Node(:FirstNode, genetics_mcr, stages_aedes, temp_none, (37.87, 122.27), migrate1)

;

Go as far as initiating the u0: this can be done in the same way for both decision and popgen models

In [ ]:
# init_node!(desired_node, gene_index)
u0 = init_node!(node, 4)
;

Extract necessary information from scenario 

In [ ]:
# Network elements 
# node_count = length(get_nodes(network))

# Genetics elements 
genetics = get_genetics(node)
gN = length(genetics.all_genotypes)
cube = genetics.cube
S = genetics.S
Τ = genetics.Τ
Β = genetics.Β
Φ = genetics.Φ
Η = genetics.Η

;

In [ ]:
# Lifestage elements 
(e, l, p, m, f) = get_allstages(node)

# Mortality 
μE = e.μ
μL = l.μ
μP = p.μ
μM = m.μ
μF = f.μ

# Substages 
nE = e.n
nL = l.n
nP = p.n
nM = m.n
nF = f.n

# Duration (only juveniles for now - though this should/might change with migration implemented)
qE = e.q
qL = l.q
qP = p.q

# Density 
densE = e.density
densL = l.density 
densP = p.density 
densM = m.density
densF = f.density 

; 

In [ ]:
l.density

### Create model
Create model object and define solver

In [ ]:
model = Model(with_optimizer(Ipopt.Optimizer))
;

### Define sets 
Where every set is a dimension, and not all the variables are necessarily subject to all the sets 

In [ ]:
# Time
T = 1:10

# Stages and substages
SE = 1:nE
SL = 1:nL
SP = 1:nP
SM = 1:nM
SF = 1:nF*gN  

# Genetics
G = 1:gN

# Nodes 
# N = 1:node_count

;

### Define variables 
Each state or control that must be calculated using a decision variable

In [ ]:
# Oviposition (produce eggsnew)
@variable(model, V[G,T] >= 0) ### Check that these sets are correct: Lacking any?

# Eggs
@variable(model, E[SE,G,T] >= 0)

# Larvae 
@variable(model, L[SL,G,T] >= 0)

# Pupae
@variable(model, P[SP,G,T] >= 0)

# Males 
@variable(model, M[SM,G,T] >= 0)

# Mating (produce matematrix)
@variable(model, X[G,T] >= 0) ### Check that these sets are correct: Need SP here?

# Females
@variable(model, F[SF,G,T] >= 0)
;

In [ ]:
E[se = [1:2], :, :]

#[g = ["hulk"],t =T[1]]

In [ ]:
#@variable(m, x_t <= 1)
#has_upper_bound(x_t)
lower_bound.(F) # have to change syntax because mine contains sets, example x_t does not. 
;

### Define constraints 
The dynamic equations of the model that employ a decision variable

In [ ]:
# Oviposition (produce eggsnew)
#@constraint(model,con, V[:,:] == sum(cube.*Τ.*S*Β.*F)) 
### Check: including sets in defining constraint "V" throws an error - why?

# Eggs
#@constraint(model,con, E[1, :, :] == V[#= should there be indices here? =#] - E[2,:,:] * (μL * densE + qE * nE))

#eggb[g = ["bruce"],t = T[2:end]],


@constraint(model,con, E[SE, G, T] ==  qE*nE*E[:,:,:] - E[:,:,:]*(μL*densE + qE*nE))

# Larvae 


# Pupae




In [ ]:
# Males 


# Mating (produce matematrix)


# Females 


